# Wolfram Alpha ReAct Agent

This notebook demonstrates how to create a custom `ReActAgent` using `Llama Index` that interacts with Wolfram Alpha. 

The agent is designed to handle various tasks, such as retrieving facts, solving equations, integrating expressions, solving word problems, etc.

<div class="alert alert-block alert-success">
  <h4>Applications</h4>
  <p>This agent is meant as a proof-of-concept rather than a useful tool on its own. For instancem this type of method could be integrated into AI Tutor for student-use to ensure factually correct responses.</p>
</div>


In [1]:
import os
import requests
import urllib.parse
import xml.etree.ElementTree as ET
from IPython.display import Markdown, display
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

def load_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def printmd(string):
    display(Markdown(string))

# Load API credentials
wolfram_api_key = load_text_file(file_path='wolfram_alpha_api_key.txt').strip("\n")
openai_api_key = load_text_file(file_path='key.txt').strip("\n")

### Define Wolfram Alpha Query Functions

> For now, we will just use the `get_info` tool, but we will implement the plotting at a later date.

In [2]:
class ExtendedWolframAlphaAPIWrapper:
    """
    An extended API wrapper for Wolfram Alpha that provides methods to fetch 
    both plot images and plaintext information for a given query.

    Attributes:
        appid (str): The application ID for Wolfram Alpha API access.
        qp (dict): The base query parameters used for each API request.
    """

    def __init__(self, appid: str):
        """
        Initializes the API wrapper with the provided application ID 
        and sets default query parameters.
        """
        self.appid = appid
        self.qp = {
            'units': 'metric',             # Use metric units for results.
            'format': 'plaintext,image',    # Request both plaintext and image formats by default.
            'primary': True,                # Ensure primary information is included.
            'appid': self.appid,            # App ID required for API authentication.
        }

    def run_plots(self, query: str) -> str:
        """
        Submits a query to Wolfram Alpha and retrieves URLs of plot images 
        related to the query. If no plot is found, returns a message indicating so.

        Parameters:
            query (str): The user query string to search for plot-related information.

        Returns:
            str: URLs of plot images if found, or an error message if not.
        """
        self.qp['format'] = 'image'      # Request image format specifically for plots.
        self.qp['podstate'] = None       # Clear specific pod state for generalized plot queries.
        self.qp['input'] = query         # Set the query input.

        # Construct API URI with encoded query parameters.
        uri = 'http://api.wolframalpha.com/v2/query?' + urllib.parse.urlencode(self.qp)
        response = requests.get(uri)

        # Return error message if the API request fails.
        if response.status_code != 200:
            return "Error: " + response.reason

        # Parse XML response to extract image URLs from 'Plot' pods.
        doc = ET.fromstring(response.text)
        plot_urls = []
        for pod in doc.findall('.//pod'):
            if pod.get('id') == 'Plot':
                for subpod in pod.findall('.//subpod'):
                    plot_urls.append(subpod.find('img').get('src'))

        # Return list of plot URLs or a message if no plots are found.
        if not plot_urls:
            return "No plot found."

        # Format the plot URLs for readability.
        plot_list = '\n'.join([f"{i+1}. {url}" for i, url in enumerate(plot_urls)])
        return f"Answer: {plot_list}"

    def get_info(self, query: str) -> str:
        """
        Submits a query to Wolfram Alpha and retrieves plaintext information 
        relevant to the query, formatted for readability.

        Parameters:
            query (str): The user query string to search for informational content.

        Returns:
            str: Structured plaintext information relevant to the query, or an error message.
        """
        self.qp['format'] = 'plaintext'  # Request plaintext format for detailed info.
        self.qp['input'] = query         # Set the query input.
        self.qp['output'] = 'json'       # Set output to JSON for easy parsing.

        # Construct API URI with encoded query parameters.
        uri = 'http://api.wolframalpha.com/v2/query?' + urllib.parse.urlencode(self.qp)
        response = requests.get(uri)

        # Return error message if the API request fails.
        if response.status_code != 200:
            return "Error: " + response.reason

        # Check if query was successful
        response = response.json()
        if not response['queryresult']['success']:
            return ('Try another query using as few words as possible to communicate the general concept. '
                    'If you are still unsuccessful, try using separate, single word queries.')
            
        # Parse JSON response and structure the extracted information.
        info = '### Wolfram Information\n'
        for pod in response['queryresult']['pods']:
            pod_info = ""
            for subpod in pod['subpods']:
                content = subpod['plaintext']
                title = subpod['title']
                if content == '':
                    continue
                if title != '':
                    pod_info += f"\t{title}:"
                pod_info += f"\t{content}\n"
            if pod_info != "":
                pod_info = f"\n{pod['title']}:\n" + pod_info
                info += pod_info

        return info

#### Run some tests

In [3]:
response = ExtendedWolframAlphaAPIWrapper(wolfram_api_key).get_info("y = 12x**2 - 3x")
print(response)

### Wolfram Information

Input:
	y = 12 x^2 - 3 x

Geometric figure:
	parabola

Alternate forms:
	y = 3 x (4 x - 1)
	-12 x^2 + 3 x + y = 0

Roots:
	x = 0
	x = 1/4

Properties as a real function:
	Domain:	R (all real numbers)
	Range:	{y element R : y>=-3/16}

Partial derivatives:
	d/dx(12 x^2 - 3 x) = 24 x - 3
	d/dy(12 x^2 - 3 x) = 0

Implicit derivatives:
	(dx(y))/(dy) = 1/(3 (-1 + 8 x))
	(dy(x))/(dx) = -3 + 24 x

Global minimum:
	min{12 x^2 - 3 x} = -3/16 at x = 1/8



In [4]:
response = ExtendedWolframAlphaAPIWrapper(wolfram_api_key).get_info("lifespan of a mosquito")
print(response)

### Wolfram Information

Input interpretation:
	mosquito | lifespan

Result:
	(9 to 62) days

Unit conversions:
	≈ 8 days 20 hours to 62 days
	≈ (730000 to 5.4×10^6) seconds
	≈ (12000 to 91000) minutes
	≈ (200 to 1500) hours
	≈ (1.2 to 9) weeks

Scientific name:
	Culicidae

Human comparisons:
	corresponding human value | 73 years
fraction of human value | (3.2×10^-4 to 0.0024)



In [5]:
response = ExtendedWolframAlphaAPIWrapper(wolfram_api_key).get_info("gravitational potential energy, 5 kg, 10 meters")
print(response)

### Wolfram Information

Input information:
	gravitational potential energy | 
mass | 5 kg (kilograms)
height | 10 meters
gravitational acceleration | 1 g (standard acceleration due to gravity on the surface of the earth)

Result:
	gravitational potential energy | 490.3 J (joules)
= 0.1362 W h (watt hours)
= 3.06×10^12 GeV (gigaelectronvolts)

Equation:
	U = m g h | 
U | gravitational potential energy
m | mass
h | height
g | gravitational acceleration
(energy of an object due to gravity near the surface of a gravitating body)



### Build the Tool for the ReAct Agent

We'll use `FunctionTool` to wrap each function and include a detailed description on how to best use the tool.

In [6]:
def create_wolfram_tools(wolfram_api_key):
    wolfram_tool = FunctionTool.from_defaults(
            fn=ExtendedWolframAlphaAPIWrapper(wolfram_api_key).get_info,
            name='wolfram_tool',
            description='''A factual information retrieval tool powered by Wolfram Alpha.
This tool is designed to provide precise answers for scientific, mathematical, 
and general knowledge queries. It can be used to retrieve definitions, calculations, and explanations for concepts. 
The query forwarded to this tool should use as few words as possible to communicate and use unambiguous whole words.
Just describe what you're interested in; you don't need to explicitly ask a question. 

Examples:

- "30% of 8 miles"
- "1/4 * (4 - 1/2)"
- "credit card balance $9000, apr 12%, $300/month"
- "If Jane has 23 cats and I have 2 cats, and then Jane gives me 5 cats, how many more cats does Jane have than I?"
- "solve x^2 + 4x + 6 = 0"
- "factor 2x^5 - 19x^4 + 58x^3 - 67x^2 + 56x - 48"
- "x+y=10, x-y=4"
- "y = 12x^2 - 3x"
- "simplify cos(arcsin(x)/2)"
- "partial fractions (x^2-4)/(x^4-x)"
- "(x^2-1)/(x^2+1)"
- "y'' + y = 0"
- "y'' + y = 0, y(0)=2, y'(0)=1"
- "differential equations sin 2x"
- "integrate x^2 sin^3 x dx"
- "integrate sin x dx from x=0 to pi"
- "d/dx x^2 y^4, d/dy x^2 y^4"
- "derivative of x^4 sin x"
- "limit (1+1/n)^n as n->infinity"
- "line through (1,2) and (2,1)"
- "work F=30N, d=100m"
- "centripetal acceleration, 30mph, 500 ft"
- "gravitational potential energy, 5 kg, 10 meters"
- "ideal gas law 2.2mol, 2.0atm, 500K"
- "photon energy 435nm"
- "H2SO4"
- "carbon"
- "10 densest elements"
- "octane + O2 -> water + CO2"
- "ocean at 300m"
- "global CO2 level"
- "nerve supply of gallbladder"
- "chromosome 5, 12.65 million base pairs"
- "photosynthesis"
- "Holy Roman Empire"
- "ww2"
- "President of Argentina"''')

    return [wolfram_tool]

### Initialize the ReAct Agent

We’ll set up the `ReActAgent` with all the operation tools, allowing it to select the appropriate tool based on the query.

In [7]:
from llama_index.core import PromptTemplate

def build_agent(wolfram_api_key, llm, verbose=True):
    # Define Wolfram Alpha tool to fetch factual information.
    wolfram_tools = create_wolfram_tools(wolfram_api_key)

    # Initialize the ReAct agent with the Wolfram Alpha tool and the LLM model.
    agent = ReActAgent.from_tools(
        tools=wolfram_tools,
        llm=llm,
        verbose=verbose,
        max_iterations=20
    )

    # Update system prompt for the agent
    react_system_header_str = """\

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. ALWAYS USE AT LEAST ONE TOOL.
Repeatedly use the tools until you have GATHERED AS MUCH FACTUAL INFORMATION as you can before completing the task at hand.
This may require breaking the task into subtasks and using different tools (or the same tool multiple times) to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if you need to.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information to answer the question without using any more tools. 
Remember to always use at least one tool before completing the task.
At that point, you MUST respond in the one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here (In the same language as the user's question)]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here (In the same language as the user's question)]
```

## Current Conversation

Below is the current conversation consisting of interleaving human and assistant messages.

    """
    react_system_prompt = PromptTemplate(react_system_header_str)

    agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
    return agent

### Initialize the Chatbot

In [8]:
class FactBasedChat:
    """
    An interactive chatbot that provides fact-based responses to student questions
    by leveraging both an LLM (Large Language Model) and the Wolfram Alpha API for
    accurate information retrieval.

    Attributes:
        message_history (list): A history of messages exchanged between the student and the tutor.
        agent (ReActAgent): The ReAct agent that processes student input and uses tools to generate responses.
    """

    def __init__(self, openai_api_key: str, wolfram_api_key: str, 
                 display_system: bool = False, verbose: bool = False):
        """
        Initializes the FactBasedChat, setting up the LLM and Wolfram Alpha tool, 
        and preparing the ReAct agent with all necessary tools.

        Parameters:
            openai_api_key (str): API key for accessing OpenAI's GPT model.
            wolfram_api_key (str): API key for accessing Wolfram Alpha's API.
            display_system (bool): If True, displays the initial system prompt (default is False).
        """
        self.display_system = display_system
        self.reset_chat()

        # Initialize the LLM with specific parameters.
        llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key, temperature=0.)

        # Initialize the ReAct agent with the Wolfram Alpha tool and the LLM model.
        self.agent =  build_agent(wolfram_api_key, llm, verbose)
        
    def reset_chat(self):
        """
        Resets the conversation history and initializes the interaction with a system prompt
        to guide the LLM in providing accurate, fact-based assistance.
        """
        self.message_history = []

        # Define the initial system prompt for the tutor's role.
        system_prompt = "## Your Role\n\n"
        system_prompt += (
            "Your role is to support the student's understanding by providing clear, accurate explanations. "
            "Use the 'wolfram_tool' to verify facts or retrieve background information on any topic as needed. "
            "This ensures that your responses are reliable and grounded in factual knowledge.\n\n"
            #"Examples of tool usage:\n"
            #'- To find factual information, use a call like wolfram_tool("lifespan of a mosquito") to retrieve details.\n'
            #'- For math help, use a function call like wolfram_tool("solve y = 12x^2 - 3x") to get equation solutions or explanations.\n\n'
            "Engage thoughtfully with the student and clarify concepts wherever possible."
        )

        # Optionally display the system prompt for debugging.
        if self.display_system:
            print(system_prompt)
        
        # Add the system prompt to the conversation history.
        self.message_history.append(ChatMessage(role="system", content=system_prompt))

    def get_response(self, student_input: str, reset: bool = False) -> str:
        """
        Processes the student's input, interacts with the LLM and tools as needed, 
        and returns the tutor's response based on the conversation history and system instructions.

        Parameters:
            student_input (str): The question or input provided by the student.
            reset (bool): If True, resets the conversation history and restarts with the system prompt.

        Returns:
            str: The tutor's response to the student's input.
        """
        if reset:
            self.reset_chat()
        
        # Generate a response using the ReAct agent.
        response = self.agent.chat(student_input, self.message_history).response

        # Append the student's input to the message history.
        self.message_history.append(ChatMessage(role="user", content=student_input))
        
        # Append the AI's response to the message history.
        self.message_history.append(ChatMessage(role="assistant", content=response))

        return response

chatbot = FactBasedChat(openai_api_key, wolfram_api_key, verbose=True)

### Test the Agent

Try different prompts to observe how the agent selects the tool and handles each type of query.

In [9]:
response = chatbot.get_response("How do I solve x^2 - 5x + 6 = 0?", reset=True)

> Running step c69dbcfd-adf4-4ebf-8ec2-372693bcb9af. Step input: How do I solve x^2 - 5x + 6 = 0?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: wolfram_tool
Action Input: {'query': 'solve x^2 - 5x + 6 = 0'}
Observation: ### Wolfram Information

Input interpretation:
	solve x^2 - 5 x + 6 = 0

Results:
	x = 2
	x = 3

Sum of roots:
	5

Product of roots:
	6

> Running step 6c469f9c-6d5b-4906-9521-350e02e1d9d5. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: To solve the equation \(x^2 - 5x + 6 = 0\), you can factor it as \((x - 2)(x - 3) = 0\). This gives the solutions \(x = 2\) and \(x = 3\).


In [11]:
response = chatbot.get_response("How much energy does a 5 kg object have if it falls from 10 meters?", reset=True)

> Running step d749d972-edb1-4d52-b032-8a4376da3c4e. Step input: How much energy does a 5 kg object have if it falls from 10 meters?
Thought: I need to calculate the gravitational potential energy of a 5 kg object falling from a height of 10 meters. I'll use the formula for gravitational potential energy, which is given by PE = m * g * h, where m is mass, g is the acceleration due to gravity, and h is height. The standard value for g is approximately 9.81 m/s².
Action: wolfram_tool
Action Input: {'query': '5 kg * 9.81 m/s^2 * 10 m'}
Observation: ### Wolfram Information

Input interpretation:
	5 kg (kilograms)×9.81 m/s^2 (meters per second squared)×10 meters

Result:
	490.5 kg m^2/s^2 (kilogram meters squared per second squared)

Unit conversions:
	490.5 N m (newton meters)
	490.5 J (joules)
	0.4905 kJ (kilojoules)
	4.905×10^9 ergs
 (unit officially deprecated)
	0.1363 W h (watt hours)

Comparisons as energy:
	 ≈ ( 0.12 ≈ 1/9 ) × energy released by explosion of one gram of TNT ( 1 g TNT

In [12]:
response = chatbot.get_response("How long did the Roman Empire last?", reset=True)

> Running step fb3a79c4-3a4f-42b6-8c81-1fce24c0beff. Step input: How long did the Roman Empire last?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: wolfram_tool
Action Input: {'query': 'duration of the Roman Empire'}
Observation: ### Wolfram Information

Input interpretation:
	Roman Empire | dates

Result:
	16-01-27 BC to 395 AD

> Running step 7b130c8e-7db8-4e96-b3b6-ed8e5b966923. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The Roman Empire lasted from 27 BC to 395 AD, which is approximately 422 years.


In [13]:
response = chatbot.get_response("Why do plants need sunlight for photosynthesis?", reset=True)

> Running step b01b8ff0-f69c-45a0-9670-3b57421e2f05. Step input: Why do plants need sunlight for photosynthesis?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: wolfram_tool
Action Input: {'query': 'photosynthesis process in plants'}
Observation: Try another query using as few words as possible to communicate the general concept. If you are still unsuccessful, try using separate, single word queries.
> Running step b669a0b5-dbae-41fe-9b96-ade7cdcf69b6. Step input: None
Thought: I need to refine my query to get the relevant information about why plants need sunlight for photosynthesis.
Action: wolfram_tool
Action Input: {'query': 'why plants need sunlight for photosynthesis'}
Observation: Try another query using as few words as possible to communicate the general concept. If you are still unsuccessful, try using separate, single word queries.
> Running step 58fd4163-9a1e-4bd2-83c7-0f11f1bfd367. Step input: None
Thought: